<a href="https://colab.research.google.com/github/farhanarrafi/proto-language-reconstruction-using-bengali-dialects/blob/main/colab/Proto_language_reconstruction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import os.path
import json


api_token = {"username":"farhanarrafi","key":"4f070f664abbd64c4eed43162a528d02"}



# https://stackoverflow.com/a/57401752/3148856
if not os.path.isdir('/root/.kaggle'):
  ! mkdir ~/.kaggle
  ! touch ~/.kaggle/kaggle.json
  with open('/root/.kaggle/kaggle.json', 'w') as file:
      json.dump(api_token, file)
  !chmod 600 ~/.kaggle/kaggle.json

#! kaggle datasets list

In [5]:
import torch
from tqdm import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
tqdm.pandas()

### Download Dataset

In [ ]:
if not os.path.isfile('BanglaDial.csv'):
    ! kaggle datasets download -d farhanarrafi/bangladial-a-dataset-for-bengali-dialect-analysis
    ! unzip bangladial-a-dataset-for-bengali-dialect-analysis.zip
    ! rm -rf bangladial-a-dataset-for-bengali-dialect-analysis.zip

Dataset URL: https://www.kaggle.com/datasets/farhanarrafi/bangladial-a-dataset-for-bengali-dialect-analysis
License(s): CC-BY-SA-4.0
Archive:  bangladial-a-dataset-for-bengali-dialect-analysis.zip
  inflating: BanglaDial.csv          
  inflating: BanglaDial.xlsx         
  inflating: BanglaDial_ A Merged and Imbalanced text Dataset for Bengali Regional dialect analysis. - Sheet1.csv  
  inflating: BanglaDial_ A Merged and Imbalanced text Dataset for Bengali Regional dialect analysis..xlsx  


In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


bangladial_dataset = pd.read_csv('BanglaDial.csv')

In [ ]:
bangladial_dataset.head()

,Sentence,Language
0,কেমন আছো ?,Standard_Bangla
1,আজকে আমার মন ভালো নেই,Standard_Bangla
2,তুমি কি করো ?,Standard_Bangla
3,এই গরমে আমার কিছু ভালো লাগে না,Standard_Bangla
4,ছেলেটি সাদা রঙয়ের একটি শার্ট পরে এসেছিল,Standard_Bangla


In [ ]:
bangladial_languages = bangladial_dataset['Language'].unique()

In [ ]:
bangladial_dataset.shape

(63303, 2)

In [ ]:
# bangladial_dataset['row'] = bangladial_dataset.groupby('Language').cumcount()

# # Pivot the table
# bangladial = bangladial_dataset.pivot(index='row', columns='Language', values='Sentence')

# # Optional: reset index if you want a clean DataFrame
# bangladial = bangladial.reset_index(drop=True)


In [ ]:
bangladial = bangladial_dataset
bangladial.head(10)

,Sentence,Language
0,কেমন আছো ?,Standard_Bangla
1,আজকে আমার মন ভালো নেই,Standard_Bangla
2,তুমি কি করো ?,Standard_Bangla
3,এই গরমে আমার কিছু ভালো লাগে না,Standard_Bangla
4,ছেলেটি সাদা রঙয়ের একটি শার্ট পরে এসেছিল,Standard_Bangla
5,মেয়েটি লাল রঙয়ের শাড়ি পরে আমার সাথে দেখা করতে ...,Standard_Bangla
6,ছেলেটি সিলেট থেকে ঢাকায় এসেছে,Standard_Bangla
7,মেয়েটি সিলেট থেকে আসা এই ছেলেটিকে অনেক ভালবাসে,Standard_Bangla
8,ছেলেটি মেয়েটাকে এখনো ভালবাসার চোখে দেখেনি,Standard_Bangla
9,মেয়েটি তাঁর সব স্বপ্নের মধ্যে ছেলেটাকে কল্পনা করে,Standard_Bangla


In [ ]:
if not os.path.isfile('Vashantor/Final/Classification.csv'):
    ! kaggle datasets download -d raiyanzannat/vashantor
    ! unzip vashantor.zip
    ! rm -rf vashantor.zip

Dataset URL: https://www.kaggle.com/datasets/raiyanzannat/vashantor
License(s): apache-2.0
Archive:  vashantor.zip
  inflating: Vashantor/Combined/Barishal_Combined_Translation.csv  
  inflating: Vashantor/Combined/Chittagong_Combined_Translation.csv  
  inflating: Vashantor/Combined/Mymensingh_Combined_Translation.csv  
  inflating: Vashantor/Combined/Noakhali_Combined_Translation.csv  
  inflating: Vashantor/Combined/Sylhet_Combined_Translation.csv  
  inflating: Vashantor/Final/Barishal.csv  
  inflating: Vashantor/Final/Chittagong.csv  
  inflating: Vashantor/Final/Classification.csv  
  inflating: Vashantor/Final/Mymensingh.csv  
  inflating: Vashantor/Final/Noakhali.csv  
  inflating: Vashantor/Final/Sylhet.csv  


In [ ]:
vashantor_barishal = pd.read_csv('Vashantor/Final/Barishal.csv')
vashantor_chittagong = pd.read_csv('Vashantor/Final/Chittagong.csv')
vashantor_noakhali = pd.read_csv('Vashantor/Final/Noakhali.csv')
vashantor_mymensingh = pd.read_csv('Vashantor/Final/Mymensingh.csv')
vashantor_sylhet = pd.read_csv('Vashantor/Final/Sylhet.csv')

In [ ]:
vashantor_barishal.columns = vashantor_barishal.columns.str.strip()
vashantor_chittagong.columns = vashantor_chittagong.columns.str.strip()
vashantor_noakhali.columns = vashantor_noakhali.columns.str.strip()
vashantor_mymensingh.columns = vashantor_mymensingh.columns.str.strip()
vashantor_sylhet.columns = vashantor_sylhet.columns.str.strip()

In [ ]:
vashantor_barishal = vashantor_barishal.dropna()
vashantor_chittagong = vashantor_chittagong.dropna()
vashantor_noakhali = vashantor_noakhali.dropna()
vashantor_mymensingh = vashantor_mymensingh.dropna()
vashantor_sylhet = vashantor_sylhet.dropna()

In [ ]:
"""
  The format of the input text MUST BE: <district> <bengali_text>
"""
vashantor_barishal['combined'] = vashantor_barishal['region_name'] + ' ' + vashantor_barishal['regional_text']
vashantor_chittagong['combined'] = vashantor_chittagong['region_name'] + ' ' + vashantor_chittagong['regional_text']
vashantor_noakhali['combined'] = vashantor_noakhali['region_name'] + ' ' + vashantor_noakhali['regional_text']
vashantor_mymensingh['combined'] = vashantor_mymensingh['region_name'] + ' ' + vashantor_mymensingh['regional_text']
vashantor_sylhet['combined'] = vashantor_sylhet['region_name'] + ' ' + vashantor_sylhet['regional_text']

In [ ]:
vashantor_chittagong.head()

,regional_text,bangla_speech,region_name,combined
0,বাংলাদেশত ৬৪ ইয়ান জেলা,বাংলাদেশে ৬৪ টা জেলা,Chittagong,Chittagong বাংলাদেশত ৬৪ ইয়ান জেলা
1,আরা বেয়াক্কুন গতহালিয়া বাইরে গেইলাম,আমরা সবাই গতকাল বাহিরে গিয়েছিলাম,Chittagong,Chittagong আরা বেয়াক্কুন গতহালিয়া বাইরে গেইলাম
2,তোইয়ার হতা বলার ধরণ বহুত সুন্দর,তোমার কথা বলার ধরন অনেক সুন্দর,Chittagong,Chittagong তোইয়ার হতা বলার ধরণ বহুত সুন্দর
3,বরিশালর মানুষ হইল্লে অয় দে?,বরিশালের মানুষ কেমন হয়?,Chittagong,Chittagong বরিশালর মানুষ হইল্লে অয় দে?
4,খুলনা জেলা কি বহুত সুন্দর নাকি?,খুলনা জেলা কি অনেক সুন্দর?,Chittagong,Chittagong খুলনা জেলা কি বহুত সুন্দর নাকি?


In [ ]:
vashantor_barishal['combined'].str.len().max()

123

In [ ]:
# https://huggingface.co/teamapocalypseml/regben2ipa-byt5small
# Load model directly
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("abdullaharean/regipa_bangla")
model = AutoModelForSeq2SeqLM.from_pretrained("abdullaharean/regipa_bangla")

In [ ]:
#text = "<district> bengali_text_here"
text = vashantor_barishal['combined'][2]
inputs = tokenizer(text, return_tensors='pt')
output_ids = model.generate(**inputs,
        max_length=128,   # or 256, depending on your text length
        num_beams=4,      # optional: makes output better (beam search)
        early_stopping=True)
print(tokenizer.decode(output_ids[0], skip_special_tokens=False))

<pad>bɐɾɔhɐl  t̪umɪ kɪ moɾe e͡ɪ̯ kɐmdɐ ko͡ɪ̯ɾɾɐ d̪et̪e pɐɾbɐ?</s>


In [ ]:
outputs = []

"""
  The format of the input text MUST BE: <district> <bengali_text>
"""
model = model.to(device)

def translate_text(text):
    inputs = tokenizer(text, return_tensors='pt').to(device)
    output_ids = model.generate(**inputs,
        max_length=128,   # or 256, depending on your text length
        num_beams=4,      # optional: makes output better (beam search)
        early_stopping=True)
    return tokenizer.decode(output_ids[0], skip_special_tokens=True)



In [ ]:
vashantor_barishal['ipa_output'] = vashantor_barishal['combined'].astype(str).apply(translate_text)

In [ ]:
vashantor_barishal.to_csv('vashantor_barishal_ipa.csv', index=False)

In [ ]:
vashantor_chittagong['ipa_output'] = vashantor_chittagong['combined'].astype(str).progress_apply(translate_text)

100%|██████████| 2499/2499 [30:11<00:00,  1.38it/s]


In [ ]:
vashantor_chittagong.to_csv('vashantor_chittagong_ipa.csv', index=False)

In [ ]:
vashantor_noakhali['ipa_output'] = vashantor_noakhali['combined'].astype(str).progress_apply(translate_text)
vashantor_noakhali.to_csv('vashantor_noakhali_ipa.csv', index=False)

100%|██████████| 2499/2499 [30:26<00:00,  1.37it/s]


In [ ]:
vashantor_mymensingh['ipa_output'] = vashantor_mymensingh['combined'].astype(str).progress_apply(translate_text)
vashantor_mymensingh.to_csv('vashantor_mymensingh_ipa.csv', index=False)

100%|██████████| 2499/2499 [30:41<00:00,  1.36it/s]


In [ ]:
vashantor_sylhet['ipa_output'] = vashantor_sylhet['combined'].astype(str).progress_apply(translate_text)
vashantor_sylhet.to_csv('vashantor_sylhet_ipa.csv', index=False)

100%|██████████| 2499/2499 [30:30<00:00,  1.36it/s]


In [ ]:
vashantor_barishal = pd.read_csv('vashantor_noakhali_ipa_with_bangla_ipa.csv')
vashantor_chittagong = pd.read_csv('vashantor_noakhali_ipa_with_bangla_ipa.csv')
vashantor_noakhali = pd.read_csv('vashantor_noakhali_ipa_with_bangla_ipa.csv')
vashantor_mymensingh = pd.read_csv('vashantor_noakhali_ipa_with_bangla_ipa.csv')
vashantor_sylhet = pd.read_csv('vashantor_noakhali_ipa_with_bangla_ipa.csv')

In [ ]:
#import epitran

# # Apply the transliteration function to the 'text' column
# df['bangla_speech_ipa'] = df['bangla_speech'].progress_apply(lambda x: epi_beng.transliterate(str(x)))

# df.to_csv('vashantor_sylhet_ipa_with_bangla_ipa.csv', index=False)